# Machine Learning Fundamentals - Lecture 03

This is the Jupyter notebook for Lecture 03 of the Machine Learning Fundamentals
course.

In [564]:
# Import the required libraries using the commonly use short names (pd, sns, ...)
import numpy as np
import pandas as pd
import seaborn as sns

# The Path object from pathlib allows us to easily build paths in an
# OS-independent fashion
from pathlib import Path

# Load the required scikit-learn classes and functions
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from scipy import stats

# Set a nicer style for Seaborn plots
sns.set_style("darkgrid")

## Part 1: load and clean the Pokémon dataset

Here we just repeat the steps already done in the previous lectures, but in a
more succint way.

In [565]:
# Load the dataset (note the use of the Path object)
df = pd.read_csv(Path("Pokemon.csv"))

# It's not good practice to have column names with spaces and other non-standard
# characters, so let's fix this by renaming the columns to standard names
df.rename(columns={
    "Type 1" : "Type1",
    "Type 2" : "Type2",
    "Sp. Atk" : "SpAtk",
    "Sp. Def" : "SpDef",
}, inplace=True)

# Replace missing values in the "Type2" column with the string "None"
df["Type2"] = df["Type2"].fillna("None")

# Since primary and secondary types are essentially categories (and not just
# strings / objects), we can convert these columns to the category type
df["Type1"] = df["Type1"].astype("category")
df["Type2"] = df["Type2"].astype("category")

Before we proceed to the interesting part, we'll perform our data scaling and
train/test data splitting.

In [566]:
# Let's use all features except the Total, which can be considered redundant
# since it's the total of the other features
features = ["HP", "Attack", "Defense", "SpAtk", "SpDef", "Speed"]

# Get only the specified features
df_X = df[features]

# Standardize them
ss = StandardScaler()
X = ss.fit_transform(df_X)

# Our labels will be the legendary status
y_leg = df["Legendary"].to_numpy()

# Let's split our data into training (80%) and test (20%) sets
# Change the random_state parameter do split data in different ways
X_train, X_test, y_train, y_test = train_test_split(X, y_leg, test_size=0.2, random_state=42)

## Part 2: Implement our own $k$-Nearest Neighbors classifier and regressor

In [567]:
# Change this variable to change k for all the tests in this section
k_for_all = 5

In [568]:
def knn_classify(X_Train, y_train, X_test, k=5):
    dists = euclidean_distances(X_test, X_Train)
    
    idx_k_min = np.argpartition(dists, k, axis=1)[:, :k]

    labels_k_min = y_train[idx_k_min]
    
    num_pred = X_test.shape[0]
    
    maj_labels = np.zeros(num_pred, dtype=y_train.dtype)

    for i, row in enumerate(labels_k_min):
        
       values, counts = np.unique(row, return_counts=True)
       
       i_max = np.argmax(counts)
       
       maj_labels[i] = values[i_max]
       
    return maj_labels

In [569]:
y_pred_ours = knn_classify(X_train, y_train, X_test, k=k_for_all)
accuracy_ours = accuracy_score(y_test, y_pred_ours)

knnClf = KNeighborsClassifier(n_neighbors=k_for_all)
knnClf.fit(X_train, y_train)
y_pred_knn = knnClf.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print(f"Accuracy (our kNN): {accuracy_ours:.4f}")
print(f"Accuracy (sklearn kNN): {accuracy_knn:.4f}")

Accuracy (our kNN): 0.9250
Accuracy (sklearn kNN): 0.9250


In [570]:
accuracy_score(y_pred_ours, y_pred_knn)

1.0

In [571]:
def knn_regression(X_Train, y_train, X_test, k=5):
    dists = euclidean_distances(X_test, X_Train)
    
    idx_k_min = np.argpartition(dists, k, axis=1)[:, :k]

    return y_train[idx_k_min].mean(axis=1)

In [572]:
y_total = df["Total"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y_total, test_size=0.2, random_state=42)

In [573]:
y_regr_ours = knn_regression(X_train, y_train, X_test, k=k_for_all)
mean_absolute_error(y_test, y_regr_ours)

13.832500000000005

In [574]:
mean_absolute_percentage_error(y_test, y_regr_ours)

0.031841346483808264

In [575]:
knnRegr = KNeighborsRegressor()
knnRegr.fit(X_train, y_train)
y_regr_knn = knnRegr.predict(X_test)
mean_absolute_error(y_test, y_regr_knn)

13.832500000000005

In [576]:
mean_absolute_percentage_error(y_test, y_regr_ours)

0.031841346483808264

In [577]:
mean_absolute_percentage_error(y_regr_ours, y_regr_knn)

0.0

# Mini-project 3 - Decision Tree classifier

In [ ]:
# -------------------------------------------
# Entropy and information gain
# -------------------------------------------

def calculate_entropy(y):
    """
    Calculates entropy for a 1D array.
    """
    vals, counts = np.unique(y, return_counts=True) # unique classes and counts
    probs = counts / counts.sum() # class probabilities
    
    return stats.entropy(probs)

def best_threshold_for_feature(X, y, feature):
    """
    Finds the best threshold for one feature.
    Midpoints between unique sorted values.
    Left branch '<=' and right '>'.
    Returns (best_threshold, best_gain).
    """
    x = X[feature].to_numpy()
    y = np.asarray(y)

    uniq = np.unique(x) # unique sorted values
    if uniq.size <= 1: # cannot split
        return None, 0.0

    midpoints = (uniq[:-1] + uniq[1:]) / 2.0
    base_e = calculate_entropy(y) # base entropy
    n = y.size

    best_thr = None
    best_gain = 0.0

    # loop over thresholds
    for thr in midpoints:
        left_mask = x <= thr
        right_mask = ~left_mask

        yl = y[left_mask]
        yr = y[right_mask]
        if yl.size == 0 or yr.size == 0:
            continue

        el = calculate_entropy(yl) # left entropy
        er = calculate_entropy(yr) # right entropy
        weighted = (yl.size / n) * el + (yr.size / n) * er
        gain = base_e - weighted

        if gain > best_gain: # keep best split
            best_gain = gain
            best_thr = thr

    return best_thr, best_gain


# -------------------------------------------
# Tree building and prediction
# -------------------------------------------

def majority_class(y):
    """
    Returns the most common label (majority vote).
    """
    vals, counts = np.unique(y, return_counts=True)
    return vals[np.argmax(counts)]

def build_tree(X, y, features, max_depth=None, depth=0):
    """
    Recursively builds a decision tree with numeric splits.
    Stops when:
    - node is pure (only one class)
    - max_depth reached
    """
    y = np.asarray(y)

    # pure node
    if np.unique(y).size == 1:
        return {"leaf": True, "class": y[0]}

    # stop rules
    if (max_depth is not None and depth >= max_depth):
        return {"leaf": True, "class": majority_class(y)}

    # single pass over features to find the best split
    best_feat = None
    best_thr = None
    best_gain = -1.0 # start below any real gain

    for feature in features:
        thr, gain = best_threshold_for_feature(X, y, feature)
        if gain > best_gain:
            best_gain = gain
            best_feat = feature
            best_thr = thr

    # no useful split
    if best_thr is None or best_gain <= 0.0:
        return {"leaf": True, "class": majority_class(y)}

    # split data
    left_mask = X[best_feat].to_numpy() <= best_thr
    right_mask = ~left_mask
    if not left_mask.any() or not right_mask.any():
        return {"leaf": True, "class": majority_class(y)}

    # grow children
    left_child = build_tree(X[left_mask], y[left_mask], features, max_depth, depth + 1)
    right_child = build_tree(X[right_mask], y[right_mask], features, max_depth, depth + 1)

    # internal node
    return {
        "leaf": False,
        "feature": best_feat,
        "threshold": float(best_thr),
        "left": left_child,
        "right": right_child
    }

def predict_one(tree, row):
    """
    Predicts one sample (row is a Series).
    Walks down the tree until a leaf.
    """
    node = tree
    while not node["leaf"]:
        feature = node["feature"]
        thr = node["threshold"]
        node = node["left"] if row[feature] <= thr else node["right"]
    return node["class"]

def predict_tree(tree, X):
    """
    Predicts all rows in X.
    Uses a list comprehension.
    """
    return np.array([predict_one(tree, X.iloc[i]) for i in range(X.shape[0])])


# -------------------------------------------
# Fit and classify
# -------------------------------------------

def fit_tree(X_train, y_train, features, max_depth=None):
    """
    Trains the tree (just calls build_tree) and returns it.
    """
    return build_tree(X_train, y_train, features, max_depth=max_depth)

def dt_classify(X_train, y_train, X_test, features, max_depth=None):
    """
    Trains on (X_train, y_train) and returns predictions for X_test.
    """
    tree = fit_tree(X_train, y_train, features, max_depth=max_depth)
    return predict_tree(tree, X_test)


# -------------------------------------------
# Text tree printer
# -------------------------------------------

def print_tree(tree, indent="", decimals=2):
    """
    Prints the tree in text. Left is '<=', right is '>'.
    """
    if tree["leaf"]:
        print(indent + "-> class:", tree["class"])
        return
    feature = tree["feature"]; thr = tree["threshold"]
    print(indent + f"[{feature} <= {thr:.{decimals}f}]")
    print_tree(tree["left"], indent + "  ", decimals=decimals)
    print(indent + f"[{feature} >  {thr:.{decimals}f}]")
    print_tree(tree["right"], indent + "  ", decimals=decimals)

In [579]:
features = ["HP", "Attack", "Defense", "SpAtk", "SpDef", "Speed"]
X = df[features]
y = df["Legendary"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Predict using our decision tree
y_pred_ours = dt_classify(X_train, y_train, X_test, features, max_depth=3)

# Compare with sklearn's DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth=3, criterion="entropy")
dt_clf.fit(X_train.values, y_train)

# Build and print our decision tree
tree = fit_tree(X_train, y_train, features, max_depth=3)
print("Our Decision Tree:")
print_tree(tree, decimals=2)

# Print the tree using sklearn's text exporter for comparison
print("\nSklearn Decision Tree:")
print(export_text(dt_clf, feature_names=features, decimals=2, show_weights=False))

# Evaluate both
print(f"Accuracy (our DT): {accuracy_score(y_test, y_pred_ours):.4f}")
print(f"Accuracy (sklearn DT): {accuracy_score(y_test, dt_clf.predict(X_test.values)):.4f}")


Our Decision Tree:
[HP <= 78.50]
  [Speed <= 114.50]
    [SpDef <= 157.00]
      -> class: False
    [SpDef >  157.00]
      -> class: False
  [Speed >  114.50]
    [SpAtk <= 87.50]
      -> class: False
    [SpAtk >  87.50]
      -> class: False
[HP >  78.50]
  [Speed <= 83.00]
    [HP <= 81.00]
      -> class: False
    [HP >  81.00]
      -> class: False
  [Speed >  83.00]
    [SpDef <= 71.00]
      -> class: False
    [SpDef >  71.00]
      -> class: True

Sklearn Decision Tree:
|--- HP <= 78.50
|   |--- Speed <= 114.50
|   |   |--- SpDef <= 157.00
|   |   |   |--- class: False
|   |   |--- SpDef >  157.00
|   |   |   |--- class: False
|   |--- Speed >  114.50
|   |   |--- SpAtk <= 87.50
|   |   |   |--- class: False
|   |   |--- SpAtk >  87.50
|   |   |   |--- class: False
|--- HP >  78.50
|   |--- Speed <= 83.00
|   |   |--- HP <= 81.00
|   |   |   |--- class: False
|   |   |--- HP >  81.00
|   |   |   |--- class: False
|   |--- Speed >  83.00
|   |   |--- SpDef <= 71.00
|   |   

# Decision Tree Classifier by "hand" (Report)

## What is a decision tree
A DT is a model that tries to make logic decisions based on question like:

- "The value of HP is lower or equal to 50 (**threshold**)?"
  - If yes, goes to the left
  - If not, goes to the right

Each **node** of the tree asks a question about a **feature**, and at the end (**on the leafs**), are the **final class** (ex.: ```"Legendary = True"``` or ```"False"```).

## Calculate Entropy

```calculate_entropy(y)```

This calculates entropy from the given y vector, that is, the mixture between classes.

- If there's only 1 class, ```entropy=0``` (Pure).
- If there's like a 50/50 between 2 classes, the ```entropy=1``` (maximum uncertainly).

``` vals, counts = np.unique(y, return_counts=True)``` - Founds the **unique classes** (ex.: ```[True, False]```) and how many times each one shows.

For example:

``` python
y = [True, False, True, False]
vals = [True, False]
counts = [0.5, 0.5]
```

```probs = counts / counts.sum()``` - Convert the counts into probabilities, on this examples the probs are ```[0.5, 0.5]```.

Then we return the entropy by using the function entropy from the scipy.stats: ```return stats.entropy(probs)```.
This function calculates how mixed the labels are based on the probabilities we just found.

## Finding the best threshold for a feature
```best_threshold_for_feature(X, y, feature)```

Finds the best cut-off point (**threshold**) in a certain **feature**, to divide the data and make classes more "pure".

For example, taking the feature ```HP```, that assigning imaginary values has ```[10, 20, 30, 40, 50]```, and the classes are, respectively, ```[False, False, True, True, True]```.

The code will attempt to cut between unique values, such as:

- Between 10 and 20 --> ```thr = 15```
- Between 20 and 30 --> ```thr = 25```
- Between 30 and 40 --> ```thr = 35```
- Between 40 and 50 --> ```thr = 45```

At each ```thr```(threshold), divide the data:
- Left: values ```<= thr```
- Right: values ```> thr```

Then it measures the **entropy before and after** the cut and calculates how much it has improved (**information gain**).

*In the code:*

``` python
x = X[feature].to_numpy()
y = np.asarray(y)
```

Convert the data into numpy arrays (to make comparison faster).

``` python
uniq = np.unique(x)
if uniq.size <= 1
    return None, 0.0
```

Removes repeated values from the feature.
If after that there’s only **one unique value**, it means all examples have the same number for this feature (for example, all ```HP = 50```).
In that case, there’s no possible threshold to split.

``` python
midpoints = (uniq[:-1] + uniq[1:]) / 2.0
```

Generate the **midpoints** between consecutive values, for example: ```[10, 20, 30] --> (10+20)/2=15, (20+30)/2=25 --> [15, 25]```

``` python
base_e = calculate_entropy(y)
```

Calculate entropy before dividing.

```
for thr in midpoints:
    left_mask = x <= thr
    right_mask = ~left_mask
```
Creates boolean masks:

- ```left_mask``` --> array of ```True/False``` indicating each one goes to the left.
- ```right_mask``` --> the inverse of the left mask

``` python
yl = y[left_mask]
yr = y[right_mask]
```
We take the classes from the left and the right.

``` python
el = calculate_entropy_y(yl)
er = calculate_entropy_y(yr)
```
And we calculate the entropy of each side.

``` python
weighted = (yl.size / n) * el + (yr.size / n) * er
gain = base_e - weighted
```

So that we can calculate the **information gain** (how much the total mixture was reduced).

``` python
if gain > best_gain:
    best_gain = gain
    best_thr = thr
```

We keep the **threshold with the best** **information gain**, and we return both (```return best_thr, best_gain```).

## Majority class
```majority_class(y)```

Chooses the **most frequent class** (in case of a draw, choose the first one).

For example: ```[True, False, False, False] --> False```. We use this when the **node** is a **leaf**.

## Building the Tree
```build_tree(X, y, features, max_depth=None, depth=0)```

Creates the tree **recursively**:

1. Checks if it is a leaf.
2. If not, chooses the best feature + threshold.
3. Divides data and repeats.

*In the code:*

``` python
if np.unique(y).size == 1:
    return {"leaf": True, "class": y[0]}
```
If all classes are equal --> pure node --> **leaf**.

``` python
if (max_depth is not None and depth >= max_depth):
    return {"leaf": True, "class": majority_class_y(y)}
```
If maximum depth has been reached --> **Stops here**.

``` python
best_feat = None
best_gain = -1.0
best_thr = None
```
Initialize the variables. ```best_gain``` starts at ```-1.0``` to ensure that any positive gain replaces it.

``` python
for feature in features:
    thr, gain = best_threshold_for_feature(X, y, feature)
    if gain > best_gain:
        best_gain = gain
        best_feat = feature
        best_thr = thr
```
Test **all the features**, see which one has the **best gain**, and keep the best one.

``` python
if best_thr is None or best_gain <= 0.0:
    return {"leaf": True, "class": majority_class(y)}
```
If there has been no improvement --> useless node --> make a leaf with the majority class.

``` python
left_mask = X[best_feat].to_numpy() <= best_thr
right_mask = ~left_mask
if not left_mask.any() or not right_mask.any():
    return {"leaf": True, "class": majority_class(y)}
```
Split the data between left and right based on the best feature and threshold.
If one side is empty --> useless node --> make a leaf with the majority class.

``` python
left_child = build_tree(X[left_mask], y[left_mask], features, max_depth, depth + 1)
right_child = build_tree(X[right_mask], y[right_mask], features, max_depth, depth + 1)
```

Call the function again (**recursion**) to build both sides of the tree.

``` python
return {
        "leaf": False,
        "feature": best_feat,
        "threshold": float(best_thr),
        "left": left_child,
        "right": right_child
    }
```
Returns the complete node.

## Predict One Sample
```predict_one(tree, row)```

Traverse a **row** of the tree until reach the final class.

For example:

``` python
[HP <= 80]
  -> if 70 <= 80 -> goes left
```

*In the code:*

``` python
while not node["leaf"]:
    feature = node["feature"]
    thr = node["threshold"]
    node = node["left"] if row[feature] <= thr else node["right"]
return node["class"]
```

## Predict Tree
```predict_tree(tree, X)```

Predicts all the rows in ```X``` using the tree.
``` python
return np.array([predict_one(tree, X.iloc[i]) for i in range(X.shape[0])])
``` 
Uses a list comprehension to call ```predict_one``` for each row. ```iloc``` is used to access rows by their integer index, and ```X.shape[0]``` gives the number of rows in the DataFrame.

## Fit Tree
```fit_tree(X_train, y_train, features, max_depth=None)```

Trains the tree using the training data.

``` python
return build_tree(X_train, y_train, features, max_depth=max_depth)
```

## Decision Tree Classify
```dt_classify(X_train, y_train, X_test, features, max_depth=None)```

Combines everything:
1. Fit the tree (```fit_tree```)
2. Predict on the test set (```predict_tree```)
3. Return the results (```array``` with True/False)

``` python
tree = fit_tree(X_train, y_train, features, max_depth=max_depth)
return predict_tree(tree, X_test)
```

## Print Tree
```print_tree(tree, indent="", decimals=2```

Prints the tree in a human-readable format. For example:

``` python
[HP <= 78.50]
  [Speed <= 114.50]
    -> class: False
  [Speed >  114.50]
    -> class: True
```